In [1]:
import keras
from keras import layers
import numpy as np

Using TensorFlow backend.


In [13]:
latent_dim = 32
height = 32
width = 32
channels = 3

generator_input = keras.Input(shape=(latent_dim,))

# Transforms the input into a 16 × 16 128-channel feature map
x = layers.Dense(128 * 16 * 16)(generator_input)
x = layers.LeakyReLU()(x)
x = layers.Reshape((16, 16, 128))(x)

x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

# Upsamples to 32 × 32
x = layers.Conv2DTranspose(256, 4, strides=2, padding='same')(x)
x = layers.LeakyReLU()(x)

x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)

# Produces a 32 × 32 1-channel feature map (shape of a CIFAR10 image)
x = layers.Conv2D(channels, 7, activation='tanh', padding='same')(x)

# Instantiates the generator model, which maps the input of shape (latent_dim,) 
#into an image of shape (32, 32, 3)
generator = keras.models.Model(generator_input, x)
generator.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 32768)             1081344   
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 32768)             0         
_________________________________________________________________
reshape_2 (Reshape)          (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 16, 16, 256)       819456    
_________________________________________________________________
leaky_re_lu_11 (LeakyReLU)   (None, 16, 16, 256)       0         
_________________________________________________________________
conv2d_transpose_2 (Conv2DTr (None, 32, 32, 256)       1048832   
__________

In [14]:
discriminator_input = layers.Input(shape=(height, width, channels))

x = layers.Conv2D(128, 3)(discriminator_input)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Flatten()(x)

# One dropout layer: an important trick
x = layers.Dropout(0.4)(x)

# Classification layer
x = layers.Dense(1, activation='sigmoid')(x)

#Instantiates the discriminator model, which turns a (32, 32, 3) input into a binary 
#classification decision (fake/real)
discriminator = keras.models.Model(discriminator_input, x)
discriminator.summary()

# Uses gradient clipping (by value) in the optimizer and To stabilize training, 
# uses learning-rate decay
discriminator_optimizer = keras.optimizers.RMSprop(lr=0.0008,clipvalue=1.0,decay=1e-8)
discriminator.compile(optimizer=discriminator_optimizer,loss='binary_crossentropy')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 30, 30, 128)       3584      
_________________________________________________________________
leaky_re_lu_15 (LeakyReLU)   (None, 30, 30, 128)       0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 14, 14, 128)       262272    
_________________________________________________________________
leaky_re_lu_16 (LeakyReLU)   (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 6, 6, 128)         262272    
_________________________________________________________________
leaky_re_lu_17 (LeakyReLU)   (None, 6, 6, 128)         0         
__________

In [0]:
# Sets discriminator weights to non-trainable (this will only apply to the gan model)
discriminator.trainable = False

gan_input = keras.Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = keras.models.Model(gan_input, gan_output)

gan_optimizer = keras.optimizers.RMSprop(lr=0.0004, clipvalue=1.0, decay=1e-8)
gan.compile(optimizer=gan_optimizer, loss='binary_crossentropy')

In [0]:
import os

os.mkdir('GAN')

In [0]:
from keras.preprocessing import image

# Loads CIFAR10 data
(x_train, y_train), (_, _) = keras.datasets.cifar10.load_data()

In [17]:
# Selects frog images (class 6)
x_train = x_train[y_train.flatten() == 6]

x_train = x_train.reshape(
    (x_train.shape[0],) +
    (height, width, channels)).astype('float32') / 255.

iterations = 10000
batch_size = 20
save_dir = 'GAN'# Specifies where you want to save generated images

start = 0
# Samples random points in the latent space
for step in range(iterations):
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))

    # Decodes them to fake images    
    generated_images = generator.predict(random_latent_vectors)

    # Combines them with real images 
    stop = start + batch_size
    real_images = x_train[start: stop]
    combined_images = np.concatenate([generated_images, real_images])

    labels = np.concatenate([np.ones((batch_size, 1)), np.zeros((batch_size, 1))])
    labels += 0.05 * np.random.random(labels.shape)

    # Trains the discriminator
    d_loss = discriminator.train_on_batch(combined_images, labels)
    
    # Samples random points in the latent space
    random_latent_vectors = np.random.normal(size=(batch_size,latent_dim))

    # Assembles latent space labels that say “these are all real images” (it’s a lie!)
    misleading_targets = np.zeros((batch_size, 1))

    # Trains the generator (via the gan model, where the discriminator weights are frozen)
    a_loss = gan.train_on_batch(random_latent_vectors, misleading_targets)

    start += batch_size
    if start > len(x_train) - batch_size: 
        start = 0
    
    if step % 100 == 0: 
        gan.save_weights('gan.h5')
    
    print('discriminator loss:', d_loss)
    print('adversarial loss:', a_loss)

    img = image.array_to_img(generated_images[0] * 255., scale=False)
    img.save(os.path.join(save_dir,
                      'generated_frog' + str(step) + '.png'))

    img = image.array_to_img(real_images[0] * 255., scale=False)
    img.save(os.path.join(save_dir,
                      'real_frog' + str(step) + '.png'))

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


discriminator loss: 0.69806415
adversarial loss: 0.7053172
discriminator loss: 4.7704144
adversarial loss: 11.424482
discriminator loss: 2.9739947
adversarial loss: 1.0000001e-07
discriminator loss: 8.427015
adversarial loss: 1.0000001e-07
discriminator loss: 8.338754
adversarial loss: 1.0000001e-07
discriminator loss: 8.295305
adversarial loss: 0.04498597
discriminator loss: 1.8026813
adversarial loss: 4.650081
discriminator loss: 1.5905588
adversarial loss: 1.8937572
discriminator loss: 0.38906783
adversarial loss: 3.2912107
discriminator loss: 1.4650377
adversarial loss: 0.77905476
discriminator loss: 0.695638
adversarial loss: 0.9008953
discriminator loss: 0.5539336
adversarial loss: 2.1210763
discriminator loss: 0.6612927
adversarial loss: 1.4723408
discriminator loss: 0.62529176
adversarial loss: 1.9071509
discriminator loss: 0.5625504
adversarial loss: 1.4653028
discriminator loss: 0.55990803
adversarial loss: 1.8918909
discriminator loss: 0.5129412
adversarial loss: 1.6052475
d

KeyboardInterrupt: ignored

In [24]:
!zip -r file.zip GAN

  adding: GAN/ (stored 0%)
  adding: GAN/generated_frog1934.png (stored 0%)
  adding: GAN/generated_frog3158.png (stored 0%)
  adding: GAN/generated_frog255.png (stored 0%)
  adding: GAN/generated_frog411.png (stored 0%)
  adding: GAN/real_frog4797.png (stored 0%)
  adding: GAN/generated_frog4628.png (stored 0%)
  adding: GAN/real_frog5329.png (stored 0%)
  adding: GAN/generated_frog3230.png (stored 0%)
  adding: GAN/real_frog1213.png (stored 0%)
  adding: GAN/generated_frog2756.png (stored 0%)
  adding: GAN/generated_frog656.png (stored 0%)
  adding: GAN/generated_frog1277.png (stored 0%)
  adding: GAN/real_frog2172.png (stored 0%)
  adding: GAN/generated_frog3605.png (stored 0%)
  adding: GAN/generated_frog1558.png (stored 0%)
  adding: GAN/real_frog6834.png (stored 0%)
  adding: GAN/generated_frog7875.png (stored 0%)
  adding: GAN/generated_frog7724.png (stored 0%)
  adding: GAN/real_frog3357.png (stored 0%)
  adding: GAN/real_frog7209.png (stored 0%)
  adding: GAN/real_frog2891.png